# S05 T01: Tasca mètodes de mostreig
***Aprèn a realitzar mostreig de les dades amb Python.***

## Nivell 1
### Exercici 1
***Agafa un conjunt de dades de tema esportiu que t'agradi. Realitza un mostreig de les dades generant una mostra aleatòria simple i una mostra sistemàtica.***


***Context

Web https://www.kaggle.com/datasets/justinas/nba-players-data?resource=download&select=all_seasons.csv


As a life-long fan of basketball I always wanted to combine my enthusiasm for the sport with passion for analytics 🏀📊. So, I utilized the NBA Stats API to pull together this data set. I hope it will prove to be as interesting to work with for you as it has been for me!

Content
The data set contains over two decades of data on each player who has been part of an NBA teams' roster. It captures demographic variables such as age, height, weight and place of birth, biographical details like the team played for, draft year and round. In addition, it has basic box score statistics such as games played, average number of points, rebounds, assists, etc.

The pull initially contained 52 rows of missing data. The gaps have been manually filled using data from Basketball Reference. I am not aware of any other data quality issues.

Analysis Ideas
The data set can be used to explore how age/height/weight tendencies have changed over time due to changes in game philosophy and player development strategies. Also, it could be interesting to see how geographically diverse the NBA is and how oversees talents have influenced it. A longitudinal study on players' career arches can also be performed.***

In [68]:
#Importem les llibreries i veiem diferents propietats
import pandas as pd
import numpy as np
import requests
import json
import re
import time
import random
import math

In [69]:
# llegim fitxer de dades
nbaPlayersDf=pd.read_csv('all_seasons.csv',index_col=False,sep=',', encoding='utf8')

In [70]:
nbaPlayersDf

,Unnamed: 0,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,0,Travis Knight,LAL,22.0,213.36,106.594120,Connecticut,USA,1996,1,...,4.8,4.5,0.5,6.2,0.127,0.182,0.142,0.536,0.052,1996-97
1,1,Matt Fish,MIA,27.0,210.82,106.594120,North Carolina-Wilmington,USA,1992,2,...,0.3,0.8,0.0,-15.1,0.143,0.267,0.265,0.333,0.000,1996-97
2,2,Matt Bullard,HOU,30.0,208.28,106.594120,Iowa,USA,Undrafted,Undrafted,...,4.5,1.6,0.9,0.9,0.016,0.115,0.151,0.535,0.099,1996-97
3,3,Marty Conlon,BOS,29.0,210.82,111.130040,Providence,USA,Undrafted,Undrafted,...,7.8,4.4,1.4,-9.0,0.083,0.152,0.167,0.542,0.101,1996-97
4,4,Martin Muursepp,DAL,22.0,205.74,106.594120,None,USA,1996,1,...,3.7,1.6,0.5,-14.5,0.109,0.118,0.233,0.482,0.114,1996-97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11695,11695,Matthew Dellavedova,CLE,30.0,190.50,90.718400,St.Mary's College of California,Australia,Undrafted,Undrafted,...,2.8,1.8,4.5,-3.1,0.029,0.085,0.125,0.312,0.337,2020-21
11696,11696,Maurice Harkless,SAC,28.0,200.66,99.790240,St. John's,USA,2012,1,...,5.2,2.4,1.2,-2.9,0.017,0.097,0.114,0.527,0.071,2020-21
11697,11697,Max Strus,MIA,25.0,195.58,97.522280,DePaul,USA,Undrafted,Undrafted,...,6.1,1.1,0.6,-4.2,0.011,0.073,0.179,0.597,0.074,2020-21
11698,11698,Marcus Morris Sr.,LAC,31.0,203.20,98.883056,Kansas,USA,2011,1,...,13.4,4.1,1.0,4.2,0.025,0.133,0.194,0.614,0.056,2020-21


In [71]:
nbaPlayersDf.describe()

,Unnamed: 0,age,player_height,player_weight,gp,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct
count,11700.000000,11700.000000,11700.000000,11700.000000,11700.000000,11700.000000,11700.000000,11700.000000,11700.000000,11700.000000,11700.000000,11700.000000,11700.000000,11700.000000
mean,5849.500000,27.131966,200.728501,100.526791,51.717179,8.169299,3.564957,1.811179,-2.166410,0.054981,0.141534,0.185380,0.510402,0.131228
std,3377.643409,4.340006,9.169827,12.526481,24.985236,5.956115,2.487498,1.792117,12.076914,0.043595,0.062793,0.052957,0.098306,0.094244
min,0.000000,18.000000,160.020000,60.327736,1.000000,0.000000,0.000000,0.000000,-200.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2924.750000,24.000000,193.040000,90.718400,32.000000,3.600000,1.800000,0.600000,-6.300000,0.021000,0.096000,0.150000,0.479000,0.065000
50%,5849.500000,26.000000,200.660000,99.790240,58.000000,6.700000,3.000000,1.200000,-1.300000,0.042000,0.132000,0.182000,0.523000,0.103000
75%,8774.250000,30.000000,208.280000,108.862080,74.000000,11.500000,4.700000,2.400000,3.200000,0.084000,0.180000,0.218000,0.559000,0.178000
max,11699.000000,44.000000,231.140000,163.293120,85.000000,36.100000,16.300000,11.700000,300.000000,1.000000,1.000000,1.000000,1.500000,1.000000


#### Mostra aleatoria simple
* Demanem grandària de la mostra aleatoria
* Definim una funció per calcular la mostra pasant el paràmetres adients ( la farem servir també per la mostra estratificada
* Es important tenir en compte que al generar numbres aleatoris podem haver duplciats, la funció te en compte aaquest fet i siselecciona dues vegades la mateixa fila genera un altre nombre

* Per control de duplkicats considerem la columna num_fila

In [72]:
def creaMostraSimple(poblacioDF,mostraSize,inici): 
    final=len(poblacioDF.index)
    columnas=poblacioDF.columns
    mostraSimple=pd.DataFrame(columns=columnas) 
    rndPos = random.randint(inici, final) 
    for x in range(mostraSize): # iterem pel valor de la mostra
        #controlem repeticions
        while poblacioDF.iloc[rndPos]['num_fila'] in mostraSimple['num_fila']:
            #si ja hi és tornem a generar un altre npmbre aleatori
            rndPos = random.randint(inici, final)
            #print("random1:"+str(rndPos))
        mostraSimple=mostraSimple.append(poblacioDF.iloc[rndPos])
    
    return mostraSimple 

In [73]:
nbaPlayersDf.rename(columns={'Unnamed: 0':'num_fila'},inplace=True)

In [74]:
numRows=len(nbaPlayersDf.index)
# Demanem el tamany del mostreig
dimensio=int(input("Entra el tamany del mostreig entre 1 i "+str(numRows)+ " : "))


Entra el tamany del mostreig entre 1 i 11700 : 234


In [75]:
mostraAleatSimpleDf=creaMostraSimple(nbaPlayersDf,dimensio,0)

In [76]:
mostraAleatSimpleDf


,num_fila,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
5310,5310,Derek Fisher,LAL,33.0,185.42,95.254320,Arkansas-Little Rock,USA,1996,1,...,11.7,2.1,2.9,6.5,0.012,0.069,0.185,0.556,0.157,2007-08
5942,5942,Marcus Landry,BOS,24.0,200.66,104.326160,Wisconsin,USA,Undrafted,Undrafted,...,2.4,1.1,0.0,0.8,0.046,0.140,0.201,0.487,0.000,2009-10
10460,10460,Anthony Davis,NOP,26.0,208.28,114.758776,Kentucky,USA,2012,1,...,25.9,12.0,3.9,3.4,0.089,0.249,0.288,0.597,0.186,2018-19
7464,7464,Brandan Wright,DAL,25.0,208.28,95.254320,North Carolina,USA,2007,1,...,8.5,4.1,0.6,5.1,0.085,0.160,0.191,0.606,0.054,2012-13
257,257,Danny Manning,PHX,31.0,208.28,106.140528,Kansas,USA,1988,1,...,13.5,6.1,2.2,3.1,0.079,0.184,0.227,0.574,0.136,1996-97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2142,2142,David Wingate,SEA,37.0,195.58,84.821704,Georgetown,USA,1986,2,...,6.0,0.0,2.0,70.6,0.000,0.000,0.140,1.000,0.286,2000-01
7745,7745,Tim Hardaway Jr.,NYK,22.0,198.12,92.986360,Michigan,USA,2013,1,...,10.2,1.5,0.8,-5.9,0.012,0.066,0.192,0.554,0.059,2013-14
8307,8307,Luis Scola,IND,35.0,205.74,108.862080,None,Argentina,2002,2,...,9.4,6.5,1.3,0.0,0.095,0.259,0.225,0.515,0.110,2014-15
8293,8293,Markel Brown,BKN,23.0,190.50,86.182480,Oklahoma State,USA,2014,2,...,4.6,2.3,0.8,-0.6,0.022,0.135,0.149,0.459,0.070,2014-15


#### Mostra sistemàtica

* Demanem al usuari el tamany de la mostra
* A partir d'aquest valor calculem la 'elevacio' , es a dir el salt que haurem de fer dese una posició a la seguent
* Definirem una funcio per crear la mostra pasant el paràmetres adients

In [77]:
def creaMostraSistematica(poblacioDF,mostraSize,inici,elevacio): 
    columnas=poblacioDF.columns
    mostraSistem=pd.DataFrame(columns=columnas) 
    numFiles=len(nbaPlayersDf.index)
    #iterem tantes vegades con la grandària de la mostra
    for x in range(mostraSize):
        inx=x+elevacio # seleccionem el element a partir de la primera posició
        mostraSistem=mostraSistem.append(poblacioDF.iloc[inx])
    return mostraSistem 

In [78]:
numFiles=len(nbaPlayersDf.index)
# Demanem el tamany del mostreig
dimMostra=int(input("Entra el tamany del mostreig entre 1 i "+str(numFiles)+ " : "))
elevacio=math.ceil(numFiles/dimMostra)
primer = round(np.random.random()*elevacio)


Entra el tamany del mostreig entre 1 i 11700 : 234


In [79]:
print(dimMostra,)
print(primer)
print(elevacio)

234
36
50


In [80]:
mostraSistematicaDf=creaMostraSistematica(nbaPlayersDf,dimMostra,primer,elevacio)

In [81]:
mostraSistematicaDf

,num_fila,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
50,50,Michael Curry,DET,28.0,195.58,95.254320,Georgia Southern,USA,Undrafted,Undrafted,...,3.9,1.5,0.5,6.7,0.024,0.094,0.120,0.592,0.062,1996-97
51,51,Michael Cage,PHI,35.0,205.74,112.490816,San Diego State,USA,1984,1,...,1.8,3.9,0.5,-12.0,0.097,0.193,0.062,0.475,0.053,1996-97
52,52,Mookie Blaylock,ATL,30.0,185.42,83.914520,Oklahoma,USA,1989,1,...,17.4,5.3,5.9,10.5,0.046,0.114,0.221,0.548,0.272,1996-97
53,53,Pooh Richardson,LAC,31.0,185.42,81.646560,UCLA,USA,1989,1,...,5.6,1.7,2.9,-5.2,0.026,0.080,0.178,0.455,0.271,1996-97
54,54,Lorenzen Wright,LAC,21.0,210.82,108.862080,Memphis,USA,1996,1,...,7.3,6.1,0.6,-3.7,0.120,0.168,0.146,0.503,0.039,1996-97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,279,Antoine Carr,UTA,35.0,205.74,115.665960,Wichita State,USA,1983,1,...,7.4,2.4,0.9,2.2,0.054,0.111,0.210,0.522,0.089,1996-97
280,280,Anthony Peeler,VAN,27.0,193.04,94.347136,Missouri,USA,1992,1,...,14.5,3.4,3.6,-10.4,0.027,0.096,0.248,0.487,0.208,1996-97
281,281,Anthony Miller,ATL,25.0,205.74,115.665960,Michigan State,USA,1994,2,...,0.0,7.0,0.0,-50.0,0.111,0.714,0.149,0.000,0.000,1996-97
282,282,Anthony Mason,CHH,30.0,203.20,113.398000,Tennessee State,USA,1988,3,...,16.2,11.4,5.7,2.6,0.074,0.239,0.179,0.585,0.216,1996-97


## Nivell 2
### Exercici 2
***Continua amb el conjunt de dades de tema esportiu i genera una mostra estratificada i una mostra utilitzant SMOTE (Synthetic Minority Oversampling Technique).***

#### Mostra estratificada
***Estratificarem la mostra per el draft_round***




***Demanem el tamany de la mostra i crearem una llista amb la quantitat del***
   
***elements del la mostra que li toca a cada grup/estrat***

***Atenció , degut als decimals pot ser que la suma de repartiments no sigui igual al total de la mostra, en aquest cas haurem de recorre la llista per ajustar.Ajustarem en funcio del tamany del estrat***


* Demanem el tamany de la mostra

In [103]:
# Total d'elements
numJugadors=len(nbaPlayersDf.index)
# Demanem el tamany del mostreig
dimMostra=int(input("Entra el tamany del mostreig entre 1 i "+str(numJugadors)+ " : "))


Entra el tamany del mostreig entre 1 i 11700 : 1170


* Creen la llista agrupada per draft_round

In [104]:
list=nbaPlayersDf.groupby(['draft_round'])['draft_round'].count()

In [105]:
list.describe

<bound method NDFrame.describe of draft_round
0               4
1            6792
2            2766
3              20
4              12
6               5
7               5
8               2
Undrafted    2094
Name: draft_round, dtype: int64>

***Convertim la list a Df i afegim una columna per saber els elements de la mostra que toca a cada draft_group***

In [106]:
listDf=pd.DataFrame(list)
listDf.rename(columns={'draft_round': 'quantitat'}, inplace=True)

In [107]:
listDf['dimround']=listDf['quantitat']*(dimMostra/numJugadors)

In [108]:
listDf['dimround']=listDf['dimround'].astype(int)


In [109]:
listDf.sort_values(by='dimround',ascending=False,inplace=True)

In [110]:
listDf

,quantitat,dimround
draft_round,,
1,6792,679
2,2766,276
Undrafted,2094,209
3,20,2
4,12,1
0,4,0
6,5,0
7,5,0
8,2,0


* Comrpovem si el repartient es correcte o cal ajustar

In [111]:
listDf

,quantitat,dimround
draft_round,,
1,6792,679
2,2766,276
Undrafted,2094,209
3,20,2
4,12,1
0,4,0
6,5,0
7,5,0
8,2,0


In [112]:
listDf['dimround'].sum()

1167

* Comprovem si el tamany de la mostra coincideix amb la suma de repartiment

In [113]:
dimRepartiment=listDf['dimround'].sum()
delta=dimMostra-dimRepartiment
pas=1 if delta > 0 else -1
for ind in range(abs(delta)):
    listDf.iat[ind,1]=listDf.iat[ind,1]+pas

    

In [114]:
if (listDf['dimround'].sum()==dimMostra):
    print("Repartiment correcte")


Repartiment correcte


* Iterem la fataframe creada per  identificar de forma ramdom els elements 
de cada llista

***Dins cada iteracio cridarem a la funcio de creació mostra simple aleatoria***

In [115]:
mostraSistemDf    = pd.DataFrame()
mostraSistemTmp= pd.DataFrame()
for row_index,row in listDf.iterrows():
    nbaPlayersDfTmp   =nbaPlayersDf[nbaPlayersDf['draft_round']==row_index]
    mostraSistemTmp   =creaMostraSimple(nbaPlayersDfTmp,row.dimround,0)
    mostraSistemDf=mostraSistemDf.append(mostraSistemTmp)

In [116]:
list=mostraSistemDf.groupby(['draft_round'])['draft_round'].count()

In [117]:
mostraSistemDf

,num_fila,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
9793,9793,Domantas Sabonis,IND,22.0,208.28,114.305184,Gonzaga,Lithuania,2016,1,...,11.6,7.7,2.0,1.4,0.093,0.227,0.214,0.567,0.127,2017-18
5630,5630,Andrew Bynum,LAL,21.0,213.36,129.273720,None,USA,2005,1,...,14.3,8.0,1.4,7.6,0.108,0.200,0.210,0.598,0.076,2008-09
8548,8548,Caron Butler,DET,35.0,200.66,103.418976,Connecticut,USA,2002,1,...,5.9,2.5,1.0,-1.8,0.014,0.123,0.132,0.544,0.074,2014-15
8971,8971,Kevin Seraphin,NYK,26.0,208.28,126.098576,Le Moyne,France,2010,1,...,3.9,2.6,1.0,-10.4,0.065,0.194,0.223,0.435,0.157,2015-16
2756,2756,Tim Thomas,MIL,26.0,208.28,108.862080,Villanova,USA,1997,1,...,13.3,4.9,1.3,-0.4,0.048,0.143,0.225,0.527,0.071,2002-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9765,9765,Gary Payton II,LAL,25.0,190.50,86.182480,Oregon State,USA,Undrafted,Undrafted,...,3.0,1.9,0.9,4.9,0.056,0.144,0.162,0.443,0.130,2017-18
10945,10945,Quinn Cook,LAL,27.0,185.42,81.646560,Duke,USA,Undrafted,Undrafted,...,5.1,1.2,1.1,4.5,0.020,0.071,0.201,0.509,0.139,2019-20
593,593,Anthony Bowie,NYK,34.0,198.12,90.718400,Oklahoma,USA,1986,3,...,2.8,1.0,0.4,-2.2,0.049,0.094,0.151,0.586,0.078,1997-98
711,711,Sam Mitchell,MIN,34.0,200.66,97.522280,Mercer,USA,1985,3,...,12.3,4.8,1.3,1.8,0.059,0.135,0.198,0.539,0.078,1997-98


In [118]:
mostraSistemDf.groupby(['draft_round'])['draft_round'].count()

draft_round
1            680
2            277
3              2
4              1
Undrafted    210
Name: draft_round, dtype: int64

In [98]:
print(dimMostra)

234


In [99]:
listDf

,quantitat,dimround
draft_round,,
1,6792,136
2,2766,56
Undrafted,2094,42
0,4,0
3,20,0
4,12,0
6,5,0
7,5,0
8,2,0


## Nivell 3
### Exercici 3
***Continua amb el conjunt de dades de tema esportiu i genera una mostra utilitzant el mètode Reservoir sampling.***


***Reservoir sampling is a family of randomized algorithms for randomly choosing k samples from a list of n items, where n is either a very large or unknown number. Typically n is large enough that the list doesn’t fit into main memory. For example, a list of search queries in Google and Facebook.
So we are given a big array (or stream) of numbers (to simplify), and we need to write an efficient function to randomly select k numbers where 1 <= k <= n. Let the input array be stream[].*** 

***A simple solution is to create an array reservoir[] of maximum size k. One by one randomly select an item from stream[0..n-1]. If the selected item is not previously selected, then put it in reservoir[]. To check if an item is previously selected or not, we need to search the item in reservoir[]. The time complexity of this algorithm will be O(k^2). This can be costly if k is big. Also, this is not efficient if the input is in the form of a stream.***

***It can be solved in O(n) time. The solution also suits well for input in the form of stream. The idea is similar to this post. Following are the steps.
***1) Create an array reservoir[0..k-1] and copy first k items of stream[] to it. 
2) Now one by one consider all items from (k+1)th item to nth item.***
…a) Generate a random number from 0 to i where i is the index of the current item in stream[]. Let the generated random number is j. 
…b) If j is in range 0 to k-1, replace reservoir[j] with stream[i]***


* Creem un llista incial amb els primers elements de la dataframe inicial corresponent al tamany de la mostra amb metode reservois sampling

In [100]:
#Definim una funcio que donat el dataframe que volem tractar i el tamny de
#la mostra , retorna dataframe amb el metode reservois sampling
def selecionaKItems(playersDf,dimMostra):
    idx=0;
    dimPlayers=len(nbaPlayersDf.index)
    # reservem una llista del tamnys de la mostra
    # on guardarem els indexs del elements seleccionats del dataframe
    reserva = [0]*dimMostra; 
    
    #empleenem incialment amb els primers elements fins al tamany de la mostra
    for idx in range(dimMostra):
        reserva[idx]=playersDf.index[idx]
    idx=dimMostra   
    
    # Recorre cada element del Df de entrada per sobra del tamany de la mostra
    while(idx < dimPlayers):
            # generem  un ramdom de 0 fins idx+1 ( posició actual del element)
            rndpos = random.randrange(idx+1);
            # Si el nombre aleatori es menor que dimMostra aleshores
            # sustituim la posició random per el item en ubicacio idx 
            # si es mes gran , aleshores 
            if(rndpos < dimMostra):
                reserva[rndpos] = playersDf.index[idx];
            idx+=1;
    
    # Creem la mostra a partir de les posicions generades
    columnas=playersDf.columns
    reservoirMostraDf=pd.DataFrame(columns=columnas)
    for elem in reserva:
        reservoirMostraDf=reservoirMostraDf.append(playersDf.iloc[elem])
    return reservoirMostraDf


In [101]:
numFiles=len(nbaPlayersDf.index)
# Demanem el tamany del mostreig
dimMostra=int(input("Entra el tamany del mostreig entre 1 i "+str(numFiles)+ " : "))

mostraReservSampolingDf=selecionaKItems(nbaPlayersDf,dimMostra)
mostraReservSampolingDf

Entra el tamany del mostreig entre 1 i 11700 : 300


,num_fila,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
3345,3345,Carlos Arroyo,UTA,24.0,187.96,91.625584,Florida International,Puerto Rico,Undrafted,Undrafted,...,12.6,2.6,5.0,-1.4,0.025,0.091,0.236,0.517,0.329,2003-04
476,476,DeJuan Wheat,MIN,24.0,182.88,74.842680,Louisville,USA,1997,2,...,1.7,0.3,0.7,-1.6,0.021,0.057,0.182,0.504,0.269,1997-98
11687,11687,Markus Howard,DEN,22.0,177.80,79.378600,Marquette,USA,Undrafted,Undrafted,...,2.8,0.6,0.5,-17.0,0.004,0.095,0.250,0.477,0.145,2020-21
6670,6670,Eric Bledsoe,LAC,21.0,185.42,88.450440,Kentucky,USA,2010,1,...,6.7,2.8,3.6,-7.0,0.045,0.098,0.179,0.499,0.254,2010-11
376,376,Ivano Newbill,ATL,26.0,208.28,111.130040,Georgia Tech,USA,1994,Undrafted,...,1.4,2.8,0.3,2.5,0.104,0.189,0.083,0.439,0.042,1996-97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3308,3308,Chris Andersen,DEN,25.0,208.28,99.790240,Blinn,USA,Undrafted,Undrafted,...,3.4,4.2,0.5,0.5,0.096,0.219,0.128,0.486,0.053,2003-04
11392,11392,Derrick Favors,UTA,29.0,205.74,120.201880,Georgia Tech,USA,2010,1,...,5.4,5.5,0.6,2.6,0.130,0.227,0.125,0.663,0.059,2020-21
1071,1071,Blue Edwards,MIA,33.0,193.04,90.718400,East Carolina,USA,1989,1,...,3.2,1.4,1.3,-1.1,0.031,0.096,0.168,0.495,0.190,1998-99
298,298,Aaron McKie,DET,24.0,195.58,94.800728,Temple,USA,1994,1,...,5.2,2.7,1.9,5.2,0.031,0.129,0.147,0.524,0.163,1996-97


In [102]:
mostraReservSampolingDf

,num_fila,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
3345,3345,Carlos Arroyo,UTA,24.0,187.96,91.625584,Florida International,Puerto Rico,Undrafted,Undrafted,...,12.6,2.6,5.0,-1.4,0.025,0.091,0.236,0.517,0.329,2003-04
476,476,DeJuan Wheat,MIN,24.0,182.88,74.842680,Louisville,USA,1997,2,...,1.7,0.3,0.7,-1.6,0.021,0.057,0.182,0.504,0.269,1997-98
11687,11687,Markus Howard,DEN,22.0,177.80,79.378600,Marquette,USA,Undrafted,Undrafted,...,2.8,0.6,0.5,-17.0,0.004,0.095,0.250,0.477,0.145,2020-21
6670,6670,Eric Bledsoe,LAC,21.0,185.42,88.450440,Kentucky,USA,2010,1,...,6.7,2.8,3.6,-7.0,0.045,0.098,0.179,0.499,0.254,2010-11
376,376,Ivano Newbill,ATL,26.0,208.28,111.130040,Georgia Tech,USA,1994,Undrafted,...,1.4,2.8,0.3,2.5,0.104,0.189,0.083,0.439,0.042,1996-97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3308,3308,Chris Andersen,DEN,25.0,208.28,99.790240,Blinn,USA,Undrafted,Undrafted,...,3.4,4.2,0.5,0.5,0.096,0.219,0.128,0.486,0.053,2003-04
11392,11392,Derrick Favors,UTA,29.0,205.74,120.201880,Georgia Tech,USA,2010,1,...,5.4,5.5,0.6,2.6,0.130,0.227,0.125,0.663,0.059,2020-21
1071,1071,Blue Edwards,MIA,33.0,193.04,90.718400,East Carolina,USA,1989,1,...,3.2,1.4,1.3,-1.1,0.031,0.096,0.168,0.495,0.190,1998-99
298,298,Aaron McKie,DET,24.0,195.58,94.800728,Temple,USA,1994,1,...,5.2,2.7,1.9,5.2,0.031,0.129,0.147,0.524,0.163,1996-97
